# JSON Path example for Headwork

We assume that we have the following table:

| Name | Type |
| --- | --- |
| path | VARCHAR(255) |
| data | JSONB |

In [30]:
from mysql.connector import connect, Error

connection = connect(
    host="localhost",
    user="user",
    password="password",
    database="headwork",
    port="3306"
)

cursor = connection.cursor()


## Using JSON_EXTRACT

MySQL's JSON PATH is limited, and conditional clauses, for example, are not directly available.

In [31]:
json_path_query = "SELECT JSON_EXTRACT(data, '$.data[*].body') FROM labelled_images WHERE path = '/images/1.dzi'"
cursor.execute(json_path_query)
result = cursor.fetchall()
print("json_path_query:", result[0])

json_path_query: ('[[{"key": "category", "value": "fan_like_plate"}, {"key": "quality", "value": 3}, {"key": "confidence", "value": 1}], [{"key": "category", "value": "multiple_irregulars_1"}, {"key": "quality", "value": 1}, {"key": "confidence", "value": 0}]]',)


## Using JSON_TABLE

In [32]:
json_table_query = """SELECT data.* 
FROM labelled_images, JSON_TABLE(labelled_images.data, '$.data[*]' COLUMNS (
  id VARCHAR(255) path '$.id',
  NESTED PATH '$.body[*]' COLUMNS (
    key_v VARCHAR(255) path '$.key', 
    value VARCHAR(255) path '$.value')
  )
) AS data"""
cursor.execute(json_table_query)
result = cursor.fetchall()
print("json_table_query:", result)

# Get average quality of images
average_quality_query = """SELECT AVG(data.value) AS average_quality
FROM labelled_images, JSON_TABLE(labelled_images.data, '$.data[*]' COLUMNS (
  id VARCHAR(255) path '$.id',
  NESTED PATH '$.body[*]' COLUMNS (
    key_v VARCHAR(255) path '$.key', 
    value VARCHAR(255) path '$.value')
  )
) AS data WHERE data.key_v = 'quality'"""
cursor.execute(average_quality_query)
result = cursor.fetchall()
print("avegage quality:", result[0])

cursor.close()

json_table_query: [('#0333e168-4092-47e6-a6e0-0e2a3b631b16', 'category', 'fan_like_plate'), ('#0333e168-4092-47e6-a6e0-0e2a3b631b16', 'quality', '3'), ('#0333e168-4092-47e6-a6e0-0e2a3b631b16', 'confidence', '1'), ('#e8957ca3-6a91-4646-8107-173046aa9eb3', 'category', 'multiple_irregulars_1'), ('#e8957ca3-6a91-4646-8107-173046aa9eb3', 'quality', '1'), ('#e8957ca3-6a91-4646-8107-173046aa9eb3', 'confidence', '0')]
avegage quality: (2.0,)


True